# 政大出國留學學校搜尋系統v1

### 想必想要出國交換的大家，看到國合處落落長的徵試學校清單，一定很難知道你/妳的成績可以去哪間學校吧!
### 於是我構想要推出一個方便大家搜尋的系統!

#### -----------------------目前功能-----------------------
#### 英語組的大家，可以用自己的英文檢定成績或是GPA來找到可以申請的學校!

#### -----------------------未來更新-----------------------
#### 1. (英語組) 因為有些學校有另行規定細項分數，所以需要另外去分細項分數來供學生篩選!
#####    例: 荷蘭的葛洛賽恩大學除了要求TOEFL iBT 80 之外， Speaking 和 Writing 還各要求至少 19 分。

#### 2. (外語組) 會加入外語組的語言成績/修課要求供篩選!
#####    已知外語組的各語言組需要提出一定程度的語言證明 and/or 修該語言相關 X 學分以上。 
#####    這部分還要去細看原始資料來判斷該怎麼抓到這些訊息。

### 資料來源: http://oic.nccu.edu.tw/oicweb4/listpartnerschool.php

## 整理資料

In [31]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

In [32]:
# 設好Column Names
column_names = ["國別","校名","甄試組別","限制錄取生交換出國時資格","締約學校交換申請截止時間",
                "語言成績要求","名額","交換期程","可交換開始學年度.學期（擇一）","獎學金補助", "費用","備註"]

In [33]:
# 讀入Data
df = pd.read_html("http://oic.nccu.edu.tw/oicweb4/listpartnerschool.php")
df = df[0]

In [34]:
#先檢視讀進來的Data
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,國別,校名,甄試組別,限制錄取生交換出國時資格,締約學校交換申請截止時間,語言成績要求,名額,交換期程,可交換開始學年度.學期（擇一）,獎學金補助,費用,備註
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,馬來西亞,馬來亞大學UNIVERSITY OF MALAYA,英語組,研究所,06-30 / 11-30,TOEFL iBT 80或IELTS 6；GPA 3/4,2,1學期或1年,109.01（一年） 109.01（一學期） 109.02（一學期）,NaN,學生出國期間依教務處註冊組學生出國選修課程實施辦法辦理學雜費等項目繳費，其餘費用自理或依兩校...,NaN
3,馬來西亞,馬來亞大學UNIVERSITY OF MALAYA,英語組,大二以上不含研究所,06-30 / 11-30,TOEFL iBT 80或IELTS 6；GPA 3/4,2,1學期或1年,109.01（一年） 109.01（一學期） 109.02（一學期）,NaN,學生出國期間依教務處註冊組學生出國選修課程實施辦法辦理學雜費等項目繳費，其餘費用自理或依兩校...,NaN
4,馬來西亞,馬來西亞國立大學NATIONAL UNIVERSITY OF MALAYSIA,英語組,大二以上不含研究所,05-30 / 10-30,TOEFL iBT 80；GPA 3/4,2,1學期或1年,109.01（一年） 109.01（一學期） 109.02（一學期）,NaN,學生出國期間依教務處註冊組學生出國選修課程實施辦法辦理學雜費等項目繳費，其餘費用自理或依兩校...,NaN


In [35]:
# 資料清理: 將不重要的費用、交換期程、備註清除，將NaN值用"無資料"取代
df.columns = column_names
df.drop([0,1], axis = 0, inplace = True)
df.drop(["費用","交換期程","備註"], axis = 1, inplace = True)
df.replace(to_replace = np.NaN, value = "無資料", inplace = True)
df.reset_index(drop = True, inplace = True)

In [36]:
# 清理過後的資料
df

,國別,校名,甄試組別,限制錄取生交換出國時資格,締約學校交換申請截止時間,語言成績要求,名額,可交換開始學年度.學期（擇一）,獎學金補助
0,馬來西亞,馬來亞大學UNIVERSITY OF MALAYA,英語組,研究所,06-30 / 11-30,TOEFL iBT 80或IELTS 6；GPA 3/4,2,109.01（一年） 109.01（一學期） 109.02（一學期）,無資料
1,馬來西亞,馬來亞大學UNIVERSITY OF MALAYA,英語組,大二以上不含研究所,06-30 / 11-30,TOEFL iBT 80或IELTS 6；GPA 3/4,2,109.01（一年） 109.01（一學期） 109.02（一學期）,無資料
2,馬來西亞,馬來西亞國立大學NATIONAL UNIVERSITY OF MALAYSIA,英語組,大二以上不含研究所,05-30 / 10-30,TOEFL iBT 80；GPA 3/4,2,109.01（一年） 109.01（一學期） 109.02（一學期）,無資料
3,香港,香港教育大學THE EDUCATION UNIVERSITY OF HONG KONG,英語組,大二以上不含研究所,04-20 / 10-15,TOEFL iBT 80或IELTS 6；GPA 2.7/4,1,109.01（一學期） 109.02（一學期）,無資料
4,香港,嶺南大學LINGNAN UNIVERSITY,英語組,大二以上不含研究所,04-30 / 10-31,TOEFL iBT 79或IELTS 6；GPA 2.5/4,2,109.01（一年） 109.01（一學期） 109.02（一學期）,無資料
...,...,...,...,...,...,...,...,...,...
332,以色列,海法大學UNIVERSITY OF HAIFA,英語組,大二以上不含研究所,05-15 / 11-15,TOEFL iBT 89,1,109.01（一年） 109.01（一學期） 109.02（一學期）,無資料
333,丹麥,南丹麥大學UNIVERSITY OF SOUTHERN DENMARK,英語組,研究所,04-01 / 10-01,TOEFL iBT 88或IELTS 6.5,1,109.01（一年） 109.01（一學期） 109.02（一學期）,無資料
334,丹麥,南丹麥大學UNIVERSITY OF SOUTHERN DENMARK,英語組,大二以上不含研究所,04-01 / 10-01,TOEFL iBT 88或IELTS 6.5,1,109.01（一年） 109.01（一學期） 109.02（一學期）,無資料
335,丹麥,奧胡斯大學AARHUS UNIVERSITY,英語組,大三以上不含研究所,05-01 / 11-01,TOEFL iBT 83或IELTS 6.5,1,109.01（一年） 109.01（一學期） 109.02（一學期）,無資料


In [37]:
#創建一個新的DataFrame，存放各項成績用
scores = pd.DataFrame(data = np.zeros((len(df),3), dtype=float), columns = ["TOEFL", "IELTS", "GPA"])

In [38]:
#這邊需要去觀察原資料集"語言成績要求"的input格式，還蠻複雜的，我使用re(RegEx)來尋找並擷取所需的成績

for i in range(len(scores)):
    if "TOEFL" in df.語言成績要求[i] :
        s = re.search("TOEFL.*[0-9]*", df.語言成績要求[i]).group()
        toefl_score = re.search("[0-9][0-9]*", s).group()
    else:
        toefl_score = 0
        
    if "IELTS" in df.語言成績要求[i]:
        s = re.search("IELTS [0-9][0-9.]*", df.語言成績要求[i]).group()
        ielts_score = re.search("[0-9][0-9.]*", s).group()
    else:
        ielts_score = 0
    
    if "GPA" in df.語言成績要求[i]:
        s = re.search("GPA [0-9.]*/[0-9]", df.語言成績要求[i]).group()
        gpa = re.search("[0-9][0-9.]*", s).group()
    else:
        gpa = 0
        
    scores.TOEFL[i] = int(toefl_score)
    scores.IELTS[i] = float(ielts_score)
    scores.GPA[i] = float(gpa)

In [39]:
#擷取過後的成績
scores

,TOEFL,IELTS,GPA
0,80.0,6.0,3.0
1,80.0,6.0,3.0
2,80.0,0.0,3.0
3,80.0,6.0,2.7
4,79.0,6.0,2.5
...,...,...,...
332,89.0,0.0,0.0
333,88.0,6.5,0.0
334,88.0,6.5,0.0
335,83.0,6.5,0.0


In [40]:
#再來就是把截出來的成績插入原先的DataFrame裡面
df.insert(6, "GPA", scores.GPA)
df.insert(6, "IELTS", scores.IELTS)
df.insert(6, "TOEFL iBT", scores.TOEFL)
# df.drop("語言成績要求", axis = 1) 可以選擇把原本的"語言成績要求"那欄去掉，但裡面可能還有重要的資訊，所以我不刪

## 搜尋系統

In [65]:
search = input("請問你想要用什麼樣的成績查詢(可複選)? ")
search_by = {}
for i in ["TOEFL iBT", "IELTS", "GPA"]:
    if i in search:
        score = input("請問你的"+i+"成績是多少? ")
        search_by[i] = float(score)
    else:
        search_by[i] = 0

print(f"以你/妳的成績可以申請{len(result)}所學校!")        
result = df[(df["TOEFL iBT"] <= search_by["TOEFL iBT"]) & (df["IELTS"] <= search_by["IELTS"]) & (df["GPA"] <= search_by["GPA"])]
pd.set_option('display.max_rows', len(result))
result.sort_values(by = ["國別", "TOEFL iBT", "IELTS", "GPA"], ascending = False)

請問你想要用什麼樣的成績查詢(可複選)? TOEFL iBT IELTS
請問你的TOEFL iBT成績是多少? 99
請問你的IELTS成績是多少? 7.5
以你/妳的成績可以申請98所學校!


,國別,校名,甄試組別,限制錄取生交換出國時資格,締約學校交換申請截止時間,語言成績要求,TOEFL iBT,IELTS,GPA,名額,可交換開始學年度.學期（擇一）,獎學金補助
5,香港,香港科技大學HONG KONG UNIVERSITY OF SCIENCE & TECHNO...,英語組,大三以上不含研究所,03-31 / 09-30,TOEFL iBT 80或IELTS 6,80.0,6.0,0.0,1,109.01（一年） 109.01（一學期） 109.02（一學期）,無資料
7,香港,香港城市大學CITY UNIVERSITY OF HONG KONG,英語組,大二以上不含研究所,04-01 / 10-02,TOEFL iBT 79或IELTS 6.5,79.0,6.5,0.0,2,109.01（一年） 109.01（一學期） 109.02（一學期）,無資料
8,香港,香港中文大學CHINESE UNIVERSITY OF HONG KONG,英語組,大二以上不含研究所,04-01 / 10-02,TOEFL iBT 71或IELTS 6,71.0,6.0,0.0,2,109.01（一年） 109.01（一學期） 109.02（一學期）,無資料
16,韓國,慶熙大學KYUNG HEE UNIVERSITY,英語組,研究所,05-30 / 11-30,TOEFL iBT 82或IELTS 6.5,82.0,6.5,0.0,1,109.01（一年） 109.01（一學期） 109.02（一學期）,無資料
17,韓國,慶熙大學KYUNG HEE UNIVERSITY,英語組,大二以上不含研究所,05-30 / 11-30,TOEFL iBT 82或IELTS 6.5,82.0,6.5,0.0,1,109.01（一年） 109.01（一學期） 109.02（一學期）,無資料
19,韓國,慶熙大學KYUNG HEE UNIVERSITY,英語組,研究所,05-30 / 11-30,TOEFL iBT 82或IELTS 6.5,82.0,6.5,0.0,1,109.01（一年） 109.01（一學期） 109.02（一學期）,無資料
20,韓國,慶熙大學KYUNG HEE UNIVERSITY,英語組,大二以上不含研究所,05-30 / 11-30,TOEFL iBT 82或IELTS 6.5,82.0,6.5,0.0,1,109.01（一年） 109.01（一學期） 109.02（一學期）,無資料
10,韓國,西江大學SOGANG UNIVERSITY,英語組,研究所或大二以上,06-10 / 12-10,TOEFL iBT 80或IELTS 6.5,80.0,6.5,0.0,1,109.01（一年） 109.01（一學期） 109.02（一年） 109.02（一學期）,無資料
15,韓國,慶北大學KYUNGPOOK NATIONAL UNIVERSITY,英語組,大二以上不含研究所,05-30 / 11-30,TOEFL iBT 80,80.0,0.0,0.0,1,109.01（一年） 109.01（一學期） 109.02（一學期）,無資料
23,韓國,仁川國立大學INCHEON NATIONAL UNIVERSITY,英語組,研究所或大二以上,05-15 / 11-15,TOEFL iBT 80,80.0,0.0,0.0,1,109.01（一年） 109.01（一學期） 109.02（一學期）,無資料
